# <font color=blue><div align="center">Seats Allocation within an Aircraft</div></font>



## Modules

In [9]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

# Module relatif à Gurobi
from gurobipy import *

# Module csv
import csv

## Préprocessing des données

Pour la suite de notre étude, on considère le modèle d'avion **Airbus A320**, qui dispose de **174** sièges. (La classe businees n'est pas prise en compte)


<img src="modele_avion.jpg" alt="Modèle de l'avion" width="400" height="600">

In [5]:
number_of_seats = 174

In [4]:
# Load the Excel file into a pandas DataFrame

df_21Oct = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=0 , skipfooter=2)
df_22Oct = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=1 , skipfooter=2)
df_23Oct = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=2 , skipfooter=2)
df_24Oct = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=3 , skipfooter=2)
df_30Oct = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=4 , skipfooter=2)
df_05Nov = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=5 , skipfooter=2)
df_07Nov = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=6 , skipfooter=2)


In [6]:
# Displaying the data

df_07Nov.head()

,Numéro du groupe,Femmes,Hommes,WCHR,TransitTime
0,1,1.0,NaN,NaN,00:55:00
1,2,NaN,1.0,NaN,00:00:00
2,3,1.0,NaN,NaN,01:15:00
3,4,1.0,1.0,NaN,03:15:00
4,5,1.0,NaN,NaN,00:00:00


In [7]:
# On choisit la df sur laquelle on va travailler

df = df_21Oct

In [33]:
# Groups : dict[int : List[int]]

Passagers = dict()
i = 1

for group in df.itertuples():
    for j in range(2,5):
        if not pd.isna(group[j]):
            for k in range(int(group[j])):
                Passagers[i] = {'gender': j-2, 'group':group[1], 'transit':group[5].hour*60 + group[5].minute }
                i+=1    
    
Passagers


{1: {'gender': 1, 'group': 1, 'transit': 75},
 2: {'gender': 0, 'group': 2, 'transit': 0},
 3: {'gender': 1, 'group': 3, 'transit': 0},
 4: {'gender': 0, 'group': 4, 'transit': 0},
 5: {'gender': 1, 'group': 5, 'transit': 325},
 6: {'gender': 1, 'group': 5, 'transit': 325},
 7: {'gender': 0, 'group': 6, 'transit': 0},
 8: {'gender': 0, 'group': 7, 'transit': 190},
 9: {'gender': 0, 'group': 8, 'transit': 80},
 10: {'gender': 1, 'group': 8, 'transit': 80},
 11: {'gender': 1, 'group': 9, 'transit': 140},
 12: {'gender': 0, 'group': 10, 'transit': 0},
 13: {'gender': 0, 'group': 11, 'transit': 235},
 14: {'gender': 1, 'group': 11, 'transit': 235},
 15: {'gender': 1, 'group': 12, 'transit': 70},
 16: {'gender': 1, 'group': 13, 'transit': 0},
 17: {'gender': 0, 'group': 14, 'transit': 0},
 18: {'gender': 1, 'group': 14, 'transit': 0},
 19: {'gender': 1, 'group': 15, 'transit': 360},
 20: {'gender': 1, 'group': 16, 'transit': 95},
 21: {'gender': 0, 'group': 17, 'transit': 60},
 22: {'gender

In [34]:
# Number of rows and columns per row based on the airplane configuration
number_of_rows = 29  # Rows are 1-indexed from 1 to 29
seats_per_row = 6    # 6 seats per row (0,1,2,3,4,5)

In [52]:
# Sample weight definitions based on group separation and proximity to the front for connecting flights

# Initialize a dictionary for weights
weights = {}

# Assign higher weights for seating group members next to each other (lower weight means higher priority)
weight_group_proximity = 1  # You might want to adjust this based on specific metrics or data

# Assign higher weights for seating passengers with connecting flights closer to the front
weight_connecting_flights = 10  # This is a simplification, you'd typically have more complex logic

# Assuming you have a way to identify group members and those with connecting flights
# Populate the weights dictionary
for i in range(1, number_of_rows + 1):
    for j in range(1, seats_per_row + 1):
        for k in Passagers:
            passenger_group = Passagers[k]['group']
            # This checks if the passenger is part of a group and assigns a weight based on proximity priority
            if Passagers[k]['group'] == passenger_group:
                weights[(i, j, k)] = weight_group_proximity
            if Passagers[k]['transit']>10:
                # Weight is inversely proportional to the row number
                # Front rows (lower i) should have lower weight
                weights[(i, j, k)] += weight_connecting_flights / i
            else:
                weights[(i, j, k)] = 0  # Default weight for regular passengers

weights

{(1, 1, 1): 11.0,
 (1, 1, 2): 0,
 (1, 1, 3): 0,
 (1, 1, 4): 0,
 (1, 1, 5): 11.0,
 (1, 1, 6): 11.0,
 (1, 1, 7): 0,
 (1, 1, 8): 11.0,
 (1, 1, 9): 11.0,
 (1, 1, 10): 11.0,
 (1, 1, 11): 11.0,
 (1, 1, 12): 0,
 (1, 1, 13): 11.0,
 (1, 1, 14): 11.0,
 (1, 1, 15): 11.0,
 (1, 1, 16): 0,
 (1, 1, 17): 0,
 (1, 1, 18): 0,
 (1, 1, 19): 11.0,
 (1, 1, 20): 11.0,
 (1, 1, 21): 11.0,
 (1, 1, 22): 11.0,
 (1, 1, 23): 11.0,
 (1, 1, 24): 0,
 (1, 1, 25): 0,
 (1, 1, 26): 0,
 (1, 1, 27): 0,
 (1, 1, 28): 11.0,
 (1, 1, 29): 11.0,
 (1, 1, 30): 11.0,
 (1, 1, 31): 11.0,
 (1, 1, 32): 0,
 (1, 1, 33): 0,
 (1, 1, 34): 11.0,
 (1, 1, 35): 11.0,
 (1, 1, 36): 11.0,
 (1, 1, 37): 0,
 (1, 1, 38): 0,
 (1, 1, 39): 11.0,
 (1, 1, 40): 0,
 (1, 1, 41): 0,
 (1, 1, 42): 0,
 (1, 1, 43): 11.0,
 (1, 1, 44): 11.0,
 (1, 1, 45): 11.0,
 (1, 1, 46): 11.0,
 (1, 1, 47): 11.0,
 (1, 1, 48): 0,
 (1, 1, 49): 11.0,
 (1, 1, 50): 11.0,
 (1, 1, 51): 11.0,
 (1, 1, 52): 11.0,
 (1, 1, 53): 11.0,
 (1, 1, 54): 0,
 (1, 1, 55): 11.0,
 (1, 1, 56): 11.0,
 (1, 1, 

In [51]:
# Initialize the model
m = Model("airplane_seating")

# Define the weights for the objective function
# Assuming weights is a dictionary mapping (i, j, k) to a weight, as discussed earlier

# Create decision variables for each seat and passenger
# The decision variable x will be a 3D dictionary with the keys being the row number,
# the seat within the row, and the unique identifier for each passenger.
x = m.addVars(range(1, number_of_rows + 1), range(1, seats_per_row + 1), Passagers.keys(), vtype=GRB.BINARY, name="x")

# Constraints
# Each seat can only be assigned to one passenger
for i in range(1, number_of_rows + 1):
    for j in range(1, seats_per_row + 1):  # Keep this range starting at 1 since you want seat numbers from 1 to 6
        m.addConstr(quicksum(x[i, j, k] for k in Passagers.keys()) <= 1, f"SeatLimit_{i}_{j}")

# Each passenger must have one seat
for k in Passagers.keys():
    m.addConstr(quicksum(x[i, j, k] for i in range(1, number_of_rows + 1) 
                         for j in range(1, seats_per_row + 1)) == 1, f"PassengerSeat_{k}")


# Additional constraints from the handwritten notes and PDF will be added here

# Objective function
# Sum the weights for each passenger-seat combination if the seat is assigned to that passenger
objective = quicksum(weights[(i, j, k)] * x[i, j, k] 
                     for i in range(1, number_of_rows + 1) 
                     for j in range(1, seats_per_row + 1) 
                     for k in Passagers.keys())
m.setObjective(objective, GRB.MINIMIZE)

# Optimize the model
m.optimize()

# Post-processing of results
# Extract the seating arrangement from the model
seating_arrangement = {(i, j): k for i in range(1, number_of_rows + 1) 
                       for j in range(1, seats_per_row + 1) 
                       for k in Passagers.keys() if x[i, j, k].X > 0.5}



Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 325 rows, 26274 columns and 52548 nonzeros
Model fingerprint: 0x496f0900
Variable types: 0 continuous, 26274 integer (26274 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 215.0087167
Presolve time: 0.04s
Presolved: 325 rows, 26274 columns, 52548 nonzeros
Variable types: 0 continuous, 26274 integer (26274 binary)
Found heuristic solution: objective 153.6604343

Root relaxation: objective 1.326055e+02, 793 iterations, 0.04 seconds (0.10 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Best

In [ ]:
# Définition du modèle
number_of_seats = 174
weight_f = 70
weight_m = 85
weight_h = 92.5
number_of_rows = 29
number_of_columns = 6

In [ ]:

Définition des Variables 

# m: Model

m = Model("Seats_Allocation")

AssignmenVarDict = {(i, j, k) : m.addVar(vtype = GRB.BINARY, name=f'passager_{i}_{j}_{k}') for i in range(1, number_of_rows + 1) for j in range(1, number_of_columns + 1) for k in range(1, len(Passagers) + 1)}
Contraintes satisfaction clients

# Contrainte : touts les passagers ont un siège 

m.addConstr(quicksum([AssignmenVarDict[(i, j, k)] for i in range(1, number_of_rows + 1) for j in range(1, number_of_columns+1) for k in range(1, len(Passagers) + 1)]) == len(Passagers), name = 'Passagers Sièges')

#Contrainte : Siège par personne

SeatPerPassengerCONST = {k : m.addConstr(quicksum([AssignmenVarDict[(i, j, k)] for i in range(1, number_of_rows + 1) for j in range(1, number_of_columns)]) == 1, name = f'Unique Siège Passager {k}') for k in range(1, len(Passagers) + 1)}
Contrainte Centrage de l'avion

PondeMen = weight_m*quicksum([AssignmenVarDict[(i,j,k)]*(i,j) for i in range(1, number_of_rows + 1) for j in range(1, number_of_columns+1) for k in range(1, len(Passagers) + 1) if Passagers[k]['gender'] == 1]) 
PondeWomen = weight_f*quicksum([AssignmenVarDict[(i,j,k)]*(i,j) for i in range(1, number_of_rows + 1) for j in range(1, number_of_columns+1) for k in range(1, len(Passagers) + 1) if Passagers[k]['gender'] == 0]) 
PondeWCHR= weight_h*quicksum([AssignmenVarDict[(i,j,k)]*(i,j) for i in range(1, number_of_rows + 1) for j in range(1, number_of_columns+1) for k in range(1, len(Passagers) + 1) if Passagers[k]['gender'] == 2]) 

Barycentre = (PondeMen + PondeWCHR + PondeWomen)/